http://vision.stanford.edu/cs598_spring07/papers/Lecun98.pdf

In [1]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import time
import cmath
import sys
import random
from mnist.loader import MNIST

In [2]:
def loadMNIST():
    mndata = MNIST('../MNIST')
    imagesTest, labelsTest = mndata.load_training()
    imagesTrain, labelsTrain = mndata.load_testing()
    return imagesTrain, labelsTrain, imagesTest, labelsTest

In [3]:
def convolve(im, kernel, bias=0):
    kernelSize = len(kernel)
    size = im.shape
    ret = (np.zeros((size[0] - 2*(kernelSize//2), size[1] - 2*(kernelSize//2))))
    for i in range(kernelSize//2, size[0] - kernelSize//2):
        for j in range(kernelSize//2, size[1] - kernelSize//2):
            ret[i - kernelSize//2][j - kernelSize//2] = np.sum(np.multiply(kernel,im[i-kernelSize//2:i-kernelSize//2+kernelSize,j-kernelSize//2:j-kernelSize//2+kernelSize]))
    ret = ret + bias
    return ret

In [4]:
def avgPool(im, k):
    ret = np.zeros(((len(im)//2), len(im[0])//2))
    for i in range(len(ret)):
        for j in range(len(ret[0])):
            ret[i][j] = np.mean(im[i*2:i*2+k, j*2:j*2+k])
    return ret

In [5]:
def reshape(x):
    return np.uint8(np.reshape(x, (int(math.sqrt(len(x))), int(math.sqrt(len(x))))))

In [6]:
def normalize(im):
    return np.where(im==0,-0.1,1.175)

In [7]:
def preprocess(im):
    return normalize(np.pad(im, (2, 2)))

In [8]:
xTrain, yTrain, xTest, yTest = loadMNIST()

In [9]:
for i in range(len(xTrain)):
    xTrain[i] = preprocess(reshape(xTrain[i]))
for i in range(len(xTest)):
    xTest[i] = preprocess(reshape(xTest[i]))

## Parameters
https://cdn.analyticsvidhya.com/wp-content/uploads/2021/03/Screenshot-from-2021-03-18-12-56-51.png

In [24]:
S2C3 = { 0: [0, 1, 2], 
         1: [1, 2, 3], 
         2: [2, 3, 4], 
         3: [3, 4, 5], 
         4: [0, 4, 5], 
         5: [0, 1, 5], 
         6: [0, 1, 2, 3], 
         7: [1, 2, 3, 4], 
         8: [2, 3, 4, 5], 
         9: [0, 3, 4, 5], 
        10: [0, 1, 4, 5], 
        11: [0, 1, 2, 5], 
        12: [0, 1, 3, 4], 
        13: [1, 2, 4, 5], 
        14: [0, 2, 3, 5], 
        15: [0, 1, 2, 3, 4, 5]}
conv = [(np.random.rand(6, 5, 5)), (np.random.rand(6, 16, 5, 5)), (np.random.rand(16, 120, 5, 5))]
bConv = [(np.random.rand(6)), (np.random.rand(16)), (np.random.rand(120))]
w1 = np.random.rand(120, 84)
b1 = np.random.rand(84)
w2 = np.random.rand(84, 10)
b2 = np.random.rand(10)

In [27]:
def forward(im):
    A = 1.7159
    # Convolution 1
    ret1 = []
    for i in range(len(conv[0])):
        ret1.append(A*np.tanh(2*convolve(im, conv[0][i], bConv[0][i])/3))
        
    # Average pooling
    for i in range(len(ret1)):
        ret1[i] = avgPool(ret1[i], 2)
        
    # Convolution 2
    ret2 = []
    for i in range(len(conv[1][0])):
        tmp = np.zeros((len(ret1[0]) - 2*(len(conv[1][0][i])//2), len(ret1[0][0]) - 2*(len(conv[1][0][i])//2)))
        for j in S2C3[i]:
            tmp += convolve(ret1[j], conv[1][j][i])
        tmp +=  bConv[1][i]
        ret2.append(A*np.tanh(2*tmp/3))
        
    # Average pooling
    for i in range(len(ret2)):
        ret2[i] = avgPool(ret2[i], 2)
        
    # Convolution 3
    ret3 = []
    for i in range(len(conv[2][0])):
        tmp = np.zeros((len(ret2[0]) - 2*(len(conv[2][0][i])//2), len(ret2[0][0]) - 2*(len(conv[2][0][i])//2)))
        for j in range(len(conv[2])):
            tmp += convolve(ret2[j], conv[2][j][i])
        tmp +=  bConv[2][i]
        ret3.append(A*np.tanh(2*tmp/3))
    
    # Fully connected 1
    ret4 = []
    for i in range(len(w1[0])):
        tmp = 0
        for j in range(len(w1)):
            tmp += w1[j][i]*ret3[j]
        tmp += b1[i]
        ret4.append(A*np.tanh(2*tmp/3))

    # Fully connected 2
    ret5 = []
    for i in range(len(w2[0])):
        tmp = 0
        for j in range(len(w2)):
            tmp += w2[j][i]*ret4[j]
        tmp += b2[i]
        ret5.append(tmp)
    
    # Softmax activation
    expSum = np.sum(np.exp(ret5))
    ret5 = np.exp(ret5)/expSum
    
    return ret5

In [29]:
forward(xTrain[0])

array([[[1.08888025e-05]],

       [[1.52597485e-05]],

       [[2.80178639e-03]],

       [[2.02280316e-01]],

       [[5.12290340e-01]],

       [[2.63636961e-05]],

       [[1.48423712e-04]],

       [[1.36188144e-03]],

       [[2.15309262e-03]],

       [[2.78911648e-01]]])

# BackProp